# Required Modules
If there are errors, please execute:
<br/>conda install numpy
<br/>conda install prettytable

In [24]:
import numpy as np
from prettytable import PrettyTable

## Import WESM Data

In [ ]:
WESM_price = np.genfromtxt('210607 RC WESM Forecast.csv',dtype=float,delimiter=',',skip_header=1)

## Please Configure Hours Charging & Discharging

In [31]:
battery_mwh = 8480
battery_mw = 2500

# Full Hour Charge
charge = np.array([3])
# Partial Charge
pcr = np.array([0.392])
# Full Hour Discharge
discharge = np.array([3])
# Partial Discharge
pdr = np.array([0.25])

start_year = 2022
end_year = 2045

#### Main Calculation Function

In [37]:
def calculatePerformanceWESMCharging(year_iter,case,activity_interval):
    i=0
    WESM_this_year = WESM_price[:,year_iter]
    charge_time = charge[case]
    discharge_time = discharge[case]
    partial_charge_rate = pcr[case]
    partial_discharge_rate = pdr[case]
    this_year = start_year + year_iter
    charge_hours = int(np.ceil(charge_time+partial_charge_rate))
    discharge_hours = int(np.ceil(discharge_time+partial_discharge_rate))
    bs_charge = 0
    bs_pcharge = 0
    bs_discharge = 0
    bs_pdischarge=0
    charge_cycles = 0
    discharge_cycles = 0
    income = 0
    cost = 0
    #print(WESM_this_year)
    battery_state = np.zeros(shape=8760, dtype=int)
    while(i!=len(battery_state)):
        try: 
            charge_window = WESM_this_year[i:i+16]
            #print(charge_window)
            cindex = (charge_window).argsort()[:charge_hours]
            #print(cindex)
            for j in range(0,len(cindex)):
                this_index = i+cindex[j]
                if j == (len(cindex)-1) and partial_charge_rate>0.0:
                    battery_state[this_index] = 2
                    bs_pcharge = bs_pcharge + 1
                    cost = cost+(WESM_this_year[this_index]*partial_charge_rate)
                else:
                    battery_state[this_index] = 1 # Charging
                    bs_charge = bs_charge+1
                    cost = cost + (WESM_this_year[this_index])
            charge_cycles = charge_cycles + 1

            #print(battery_state[0:24])
            i = i+np.amax(cindex)+1
            #print(i)

            discharge_window = WESM_this_year[i:i+activity_interval]
            #print(discharge_window)
            dindex = (-discharge_window).argsort()[:discharge_hours]
            #print(dindex)
            for j in range(0,len(dindex)):
                this_index = i+dindex[j]
                if j == (len(dindex)-1) and partial_discharge_rate>0.0:
                    battery_state[this_index] = 4
                    bs_pdischarge = bs_pdischarge + 1
                    income = income+(WESM_this_year[this_index]*partial_discharge_rate)
                else:
                    battery_state[this_index] = 3 # Discharging
                    bs_discharge = bs_discharge + 1
                    income = income + (WESM_this_year[this_index])
            #print(battery_state[0:24])
            discharge_cycles = discharge_cycles+1
            i = i+np.amax(dindex)+1
        #print(i)
        except Exception as e:
            #print(i,'//',e)
            break
    count_hours = np.bincount(battery_state)
    hours_charging = bs_charge + partial_charge_rate*bs_pcharge
    hours_discharging = bs_discharge + partial_discharge_rate*bs_pdischarge
    hours_idle = 8760 - (hours_charging+hours_discharging)
    profit = income - cost
    results = np.array([this_year,(charge_time+partial_charge_rate),(discharge_time+partial_discharge_rate),activity_interval,hours_charging, hours_discharging,hours_idle,charge_cycles,discharge_cycles,round(cost,2),round(income,2),round(profit,2)])
    return results


In [36]:
x = PrettyTable()
x.field_names = ["Year","Charge Time","Discharge Time","Activity Interval","Hours Charging","Hours Discharging","Hours Idle","Charge Cycles","Discharge Cycles","Cost","Income","Profit"]
for i in range(0,(end_year-start_year+1)):
    result = calculatePerformanceWESMCharging(i,0,16)
    x.add_row(result)
print(x)

+--------+-------------+----------------+-------------------+----------------+-------------------+------------+---------------+------------------+---------+---------+---------+
|  Year  | Charge Time | Discharge Time | Activity Interval | Hours Charging | Hours Discharging | Hours Idle | Charge Cycles | Discharge Cycles |   Cost  |  Income |  Profit |
+--------+-------------+----------------+-------------------+----------------+-------------------+------------+---------------+------------------+---------+---------+---------+
| 2022.0 |    3.392    |      3.25      |        16.0       |    1241.472    |      1186.25      |  6332.278  |     366.0     |      366.0       | 2451.84 |  6569.9 | 4118.06 |
| 2023.0 |    3.392    |      3.25      |        16.0       |    1241.472    |      1186.25      |  6332.278  |     366.0     |      366.0       | 2295.49 | 7399.74 | 5104.25 |
| 2024.0 |    3.392    |      3.25      |        16.0       |    1241.472    |      1186.25      |  6332.278  |    